<a href="https://colab.research.google.com/github/Pavlos01232/Match_Outcome_Prediction/blob/main/Game_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:

import torch as t
import torch.nn as n
import pandas as pd
import os

print("beep boop")
print("Aston Villa loses")
print("=============================================================")
#GITHUB LOCATION:
#https://github.com/Pavlos01232/Match_Outcome_Prediction
#
#
#
#
#
#
#




#pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0304.csv?raw=true')


#"DEEP learning" just means "hidden" layers



#read function
#df = pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0405.csv?raw=true',sep='\t', lineterminator='\r')
#print(df)

first = "https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL"
last = ".csv?raw=true"
#3 to 23
#the first two files in the training data are formatted incorrectly
#convert csv to dataframe



'''
Starting with an array of data frames,
and an array of column names, make a
single array with the chosen columns
'''




def combine(dataFrames,columnNames):
 t=[]
 for i in range(0,len(dataFrames)):
    theseColumns=dataFrames[i].columns.values[0].split(",")
    for j in range(0 ,len(dataFrames[i])):
      row=dataFrames[i].values[j][0].split(",")
      newEntry=[]
      for k in range(0, len(columnNames)):
         for m in range(0, len(theseColumns)):
             if(columnNames[k]==theseColumns[m] and m<=len(row)):
                newEntry.append(row[m])
      t.append(newEntry)
 return t





'''
Load data frames into the array df
'''


df=[]

for i in range(5, 7):
  result = first + str('{:02.0f}'.format(i)) + str('{:02.0f}'.format(i+1)) + last
  x = pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r')
  df.append(x)




'''
Get an array with each (home) team listed once
from an array of data frames with column "HomeTeam"
'''





def getTeams(df):
 teams=[]
 homeTeams=combine(df,["HomeTeam"])
 for i in range(len(homeTeams)):
  if(len(homeTeams[i])>0):
   found=False
   for j in range(len(teams)):
    if homeTeams[i][0] == teams[j]:
      found=True
      break
   if found:
    continue
   teams.append(homeTeams[i][0])
 return teams


'''
Get the list of teams from the array of data frames called df
and print it to the console
'''


teams=getTeams(df)
print(teams)





'''
Create an array called Data with just the team names and scores
from the data framees in the array of frames df, and print it
'''



Data=combine(df, ["HomeTeam","AwayTeam","FTHG","FTAG"])
print(Data)

#df.to_csv(r'C:\Users\Pavlos\Desktop\export_dataframe.csv', sep='\t', encoding='utf-8')
#print (df[2])
#file_list = os.listdir('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/')


'''
Get the numerical index of a team name x in the array teams
otherwise just return x
'''

def getIndex(x,teams):
  for i in range(len(teams)):
   if(teams[i]==x):
    return i
  return x



print(getIndex("Everton",teams))


'''
Replace any occurrence of names from the array teams
which occur anywhere in A by their actual  numbers
'''

def teamsToNumbers(A,teams):
  B=[]
  for i in range(len(A)):
    B.append([])
    for j in range(len(A[i])):
      B[i].append(getIndex(A[i][j],teams))
  return B


'''
Create Data2 which is a copy of Data but with team names
replaced by their index
'''

Data2=teamsToNumbers(Data,teams)
print(Data2)


'''


Assume Data2 has team indices in column 0 and 1 and scores
in cols 2 and 3

A[i] is array of average goals landed per game by tean i
B[i] is array of average goals conceded per game by team i
C is total games played by all teams (twide the number of games)
games[i]=total games played by team i
a[i]*b[j]=first approx of expected goals landed by i when playing
    against j


A=[0]*len(teams)
B=[0]*len(teams)
games=[0]*len(teams)
a=[0]*len(teams)
b=[0]*len(teams)
C=0
totalGames=0

for i in range(len(Data2)):

  games[Data2[i][0]]+=1
  games[Data2[i][1]]+=1
  A[Data2[i][0]]+=int(Data2[i][2])
  B[Data2[i][0]]+=int(Data2[i][3])
  A[Data2[i][1]]+=int(Data2[i][3])
  B[Data2[i][1]]+=int(Data2[i][2])
  C+=int(Data2[i][0])+int(Data2[i][1])
  totalGames+=2



'''
print("=============================================================")

i=5

stats=t.tensor(i)

#stats[0]=5
#stats[1]=2
#stats[2]=8
#stats[3]=4
#stats[4]=8
#stats[5]=3

print(stats)

t.cuda.memory_allocated(0)/1024**3

beep boop
Aston Villa loses
['Aston Villa', 'Everton', 'Fulham', 'Man City', 'Middlesbrough', 'Portsmouth', 'Sunderland', 'West Ham', 'Arsenal', 'Wigan', 'Birmingham', 'Blackburn', 'Charlton', 'Liverpool', 'Man United', 'Newcastle', 'Tottenham', 'West Brom', 'Bolton', 'Chelsea', 'Reading', 'Sheffield United', 'Watford']
[['Aston Villa', 'Bolton', '2', '2'], ['Everton', 'Man United', '0', '2'], ['Fulham', 'Birmingham', '0', '0'], ['Man City', 'West Brom', '0', '0'], ['Middlesbrough', 'Liverpool', '0', '0'], ['Portsmouth', 'Tottenham', '0', '2'], ['Sunderland', 'Charlton', '1', '3'], ['West Ham', 'Blackburn', '3', '1'], ['Arsenal', 'Newcastle', '2', '0'], ['Wigan', 'Chelsea', '0', '1'], ['Birmingham', 'Man City', '1', '2'], ['Blackburn', 'Fulham', '2', '1'], ['Charlton', 'Wigan', '1', '0'], ['Liverpool', 'Sunderland', '1', '0'], ['Man United', 'Aston Villa', '1', '0'], ['Newcastle', 'West Ham', '0', '0'], ['Tottenham', 'Middlesbrough', '2', '0'], ['West Brom', 'Portsmouth', '2', '1'], ['

IndexError: list index out of range

In [ ]:
C